In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

# -------------------------
# 0) Paths (robust if you run from /notebooks)
# -------------------------
cwd = Path.cwd()
PROJECT_ROOT = cwd.parent if cwd.name.lower() == "notebooks" else cwd

DATA_PROCESSED = PROJECT_ROOT / "data_processed"
OUT_DIR = DATA_PROCESSED / "sql_model"
OUT_DIR.mkdir(parents=True, exist_ok=True)

MASTER_PATH = DATA_PROCESSED / "pl_master_21-25_v1.csv"
if not MASTER_PATH.exists():
    raise FileNotFoundError(f"Master file not found: {MASTER_PATH.resolve()}")

# -------------------------
# 1) Load master
# -------------------------
df = pd.read_csv(MASTER_PATH)

# Ensure Date is parsed consistently (your data is already yyyy-mm-dd, this just makes it explicit)
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d", errors="coerce")

# -------------------------
# 2) dim_season
# -------------------------
seasons = sorted(df["Season"].dropna().unique())
dim_season = pd.DataFrame({"season": seasons})
dim_season["season_id"] = range(1, len(dim_season) + 1)
dim_season = dim_season[["season_id", "season"]]

season_map = dict(zip(dim_season["season"], dim_season["season_id"]))

# -------------------------
# 3) dim_team (union across all relevant team columns)
# -------------------------
team_cols = ["Team", "home_team", "away_team", "opponent"]
all_teams = pd.unique(pd.concat([df[c].dropna().astype(str) for c in team_cols], ignore_index=True))
all_teams = sorted(all_teams)

dim_team = pd.DataFrame({"team_name": all_teams})
dim_team["team_id"] = range(1, len(dim_team) + 1)
dim_team = dim_team[["team_id", "team_name"]]

team_map = dict(zip(dim_team["team_name"], dim_team["team_id"]))

# -------------------------
# 4) dim_match (1 row per match_id)
# -------------------------
dim_match = (
    df[["match_id", "Season", "Date", "home_team", "away_team", "home_goals", "away_goals"]]
    .drop_duplicates("match_id")
    .copy()
)

dim_match["season_id"] = dim_match["Season"].map(season_map).astype("int64")
dim_match["home_team_id"] = dim_match["home_team"].astype(str).map(team_map).astype("int64")
dim_match["away_team_id"] = dim_match["away_team"].astype(str).map(team_map).astype("int64")

dim_match["match_date"] = dim_match["Date"].dt.date
dim_match["match_label"] = (
    dim_match["home_team"].astype(str)
    + " "
    + dim_match["home_goals"].astype(str)
    + "-"
    + dim_match["away_goals"].astype(str)
    + " "
    + dim_match["away_team"].astype(str)
)

dim_match = dim_match[
    ["match_id", "season_id", "match_date", "home_team_id", "away_team_id", "home_goals", "away_goals", "match_label"]
].sort_values(["season_id", "match_date", "match_id"])

# -------------------------
# 5) fact_team_match (1 row per team per match)
# -------------------------
fact = df.copy()

fact["season_id"] = fact["Season"].map(season_map).astype("int64")
fact["team_id"] = fact["Team"].astype(str).map(team_map).astype("int64")
fact["opponent_team_id"] = fact["opponent"].astype(str).map(team_map).astype("int64")

# Convert boolean to int (SQL-friendly)
fact["is_home"] = fact["is_home"].astype(int)

# Drop redundant string columns that will live in dimensions
# (keep match_id + keys + metrics)
drop_cols = ["Season", "Team", "opponent", "home_team", "away_team", "Match", "Date"]
fact = fact.drop(columns=[c for c in drop_cols if c in fact.columns])

# Reorder: keys first
key_cols = ["match_id", "season_id", "team_id", "opponent_team_id", "is_home"]
other_cols = [c for c in fact.columns if c not in key_cols]
fact = fact[key_cols + other_cols]

# -------------------------
# 6) Quick validations vs master expectations
# -------------------------
assert len(df) == 3040, f"Expected 3040 rows in master, got {len(df)}"
assert len(dim_match) == 1520, f"Expected 1520 unique matches, got {len(dim_match)}"
assert len(fact) == 3040, f"Fact should have same rows as master, got {len(fact)}"
assert fact[["match_id", "team_id"]].duplicated().sum() == 0, "Duplicate (match_id, team_id) found in fact!"

# -------------------------
# 7) Save outputs (CSV) to load into SQL easily
# -------------------------
dim_season.to_csv(OUT_DIR / "dim_season.csv", index=False, encoding="utf-8")
dim_team.to_csv(OUT_DIR / "dim_team.csv", index=False, encoding="utf-8")
dim_match.to_csv(OUT_DIR / "dim_match.csv", index=False, encoding="utf-8")
fact.to_csv(OUT_DIR / "fact_team_match.csv", index=False, encoding="utf-8")

print("✅ Saved:")
print(" -", (OUT_DIR / "dim_season.csv").resolve())
print(" -", (OUT_DIR / "dim_team.csv").resolve())
print(" -", (OUT_DIR / "dim_match.csv").resolve())
print(" -", (OUT_DIR / "fact_team_match.csv").resolve())

print("\nCounts:")
print("dim_season:", len(dim_season))
print("dim_team:", len(dim_team))
print("dim_match:", len(dim_match))
print("fact_team_match:", len(fact))


✅ Saved:
 - C:\Users\tozes\Documents\IronHack\pl_21-25_analysis\data_processed\sql_model\dim_season.csv
 - C:\Users\tozes\Documents\IronHack\pl_21-25_analysis\data_processed\sql_model\dim_team.csv
 - C:\Users\tozes\Documents\IronHack\pl_21-25_analysis\data_processed\sql_model\dim_match.csv
 - C:\Users\tozes\Documents\IronHack\pl_21-25_analysis\data_processed\sql_model\fact_team_match.csv

Counts:
dim_season: 4
dim_team: 26
dim_match: 1520
fact_team_match: 3040


In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

# --- paths (assumindo notebook em /notebooks) ---
csv_path = Path("..") / "data_processed" / "sql_model" / "fact_team_match.csv"
df = pd.read_csv(csv_path)

def max_len(s: pd.Series) -> int:
    s = s.dropna().astype(str)
    return int(s.map(len).max()) if len(s) else 1

def mysql_type(col: str, s: pd.Series) -> str:
    # regras "fortes" primeiro
    if col == "match_id":
        return "VARCHAR(120)"
    if col in {"result"}:
        return "CHAR(1)"
    if col in {"is_home"}:
        return "TINYINT"
    if col.lower().endswith("_id"):
        return "INT"
    if col.lower() == "date":
        return "DATE"

    # inferência por dtype
    if pd.api.types.is_integer_dtype(s):
        return "INT"
    if pd.api.types.is_float_dtype(s):
        return "DOUBLE"

    # strings
    l = max_len(s)
    return f"VARCHAR({min(max(l, 1), 255)})" if l <= 255 else "TEXT"

# colunas NOT NULL (chaves)
not_null_cols = {"match_id", "season_id", "team_id", "opponent_team_id", "is_home"}

lines = []
for col in df.columns:
    col_sql = f"`{col}`"  # backticks para nomes com espaços/pontos
    t = mysql_type(col, df[col])
    nn = " NOT NULL" if col in not_null_cols else ""
    lines.append(f"  {col_sql} {t}{nn}")

ddl = f"""
DROP TABLE IF EXISTS `fact_team_match`;

CREATE TABLE `fact_team_match` (
{",\n".join(lines)},
  PRIMARY KEY (`match_id`, `team_id`),
  INDEX `idx_ftm_season` (`season_id`),
  INDEX `idx_ftm_team` (`team_id`),
  INDEX `idx_ftm_opponent` (`opponent_team_id`),
  CONSTRAINT `fk_ftm_match` FOREIGN KEY (`match_id`) REFERENCES `dim_match`(`match_id`),
  CONSTRAINT `fk_ftm_season` FOREIGN KEY (`season_id`) REFERENCES `dim_season`(`season_id`),
  CONSTRAINT `fk_ftm_team` FOREIGN KEY (`team_id`) REFERENCES `dim_team`(`team_id`),
  CONSTRAINT `fk_ftm_opponent` FOREIGN KEY (`opponent_team_id`) REFERENCES `dim_team`(`team_id`)
) ENGINE=InnoDB;
""".strip()

print(ddl)


DROP TABLE IF EXISTS `fact_team_match`;

CREATE TABLE `fact_team_match` (
  `match_id` VARCHAR(120) NOT NULL,
  `season_id` INT NOT NULL,
  `team_id` INT NOT NULL,
  `opponent_team_id` INT NOT NULL,
  `is_home` TINYINT NOT NULL,
  `xG` DOUBLE,
  `xGA` DOUBLE,
  `xGD` DOUBLE,
  `Open Play xG` DOUBLE,
  `Open Play xGA` DOUBLE,
  `Open Play xGD` DOUBLE,
  `Set Piece xG` DOUBLE,
  `Set Piece xGA` DOUBLE,
  `Set Piece xGD` DOUBLE,
  `npxG` DOUBLE,
  `npxGA` DOUBLE,
  `npxGD` DOUBLE,
  `Goals` INT,
  `Goals Conceded` INT,
  `GD` INT,
  `GD-xGD` DOUBLE,
  `Possession` DOUBLE,
  `Field Tilt` DOUBLE,
  `Avg Pass Height` DOUBLE,
  `xT` DOUBLE,
  `xT Against` DOUBLE,
  `Passes in Opposition Half` INT,
  `Passes into Box` INT,
  `Shots` INT,
  `Shots Faced` INT,
  `Shots per 1.0 xT` DOUBLE,
  `Shots Faced per 1.0 xT Against` DOUBLE,
  `PPDA` DOUBLE,
  `High Recoveries` INT,
  `High Recoveries Against` INT,
  `Crosses` INT,
  `Corners` INT,
  `Fouls` INT,
  `On-Ball Pressure` DOUBLE,
  `On-Ball Pre